In [1]:
import os
import cv2
import numpy as np
from albumentations import (
    Compose, RandomRotate90, HorizontalFlip, VerticalFlip, Transpose,
    Affine, RandomBrightnessContrast,
    GaussNoise, ElasticTransform, CoarseDropout
)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
aug_pipeline = Compose([
    RandomRotate90(p=0.5),
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    Transpose(p=0.2),
    Affine(rotate=(-15, 15), scale=(0.9, 1.1), translate_percent=(0.0, 0.1), p=0.5),
    RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    GaussNoise(std_range=(0.2, 0.44), per_channel=True, p=0.3),
    ElasticTransform(alpha=1, sigma=50, p=0.2),
    CoarseDropout(
        num_holes_range=(1, 8),
        hole_height_range=(0.05, 0.15),
        hole_width_range=(0.05, 0.15),
        fill=0,
        p=0.3
    )
])

In [3]:
def augment_and_save(train_dir, out_dir, augmentations_per_image=5):
    """
    Reads images from train_dir/<class> folders, applies augmentations,
    and writes original + augmented images to out_dir/<class>.
    """
    os.makedirs(out_dir, exist_ok=True)
    classes = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]

    for cls in classes:
        src_cls = os.path.join(train_dir, cls)
        dst_cls = os.path.join(out_dir, cls)
        os.makedirs(dst_cls, exist_ok=True)

        for img_name in os.listdir(src_cls):
            img_path = os.path.join(src_cls, img_name)
            # Read & convert to RGB
            img_bgr = cv2.imread(img_path)
            if img_bgr is None:
                continue
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            base, ext = os.path.splitext(img_name)

            # Save original
            cv2.imwrite(os.path.join(dst_cls, f"{base}_orig{ext}"), img_bgr)

            # Save augmentations
            for i in range(augmentations_per_image):
                aug = aug_pipeline(image=img_rgb)["image"]
                aug_bgr = cv2.cvtColor(aug, cv2.COLOR_RGB2BGR)
                cv2.imwrite(os.path.join(dst_cls, f"{base}_aug{i}{ext}"), aug_bgr)

In [4]:
orig_data_dir = "/content/drive/MyDrive/Colab Notebooks/Covid19-dataset/train"
augmented_dir  = "/content/drive/MyDrive/Colab Notebooks/Covid19-dataset/augmented"
augment_and_save(orig_data_dir, augmented_dir, augmentations_per_image=5)